<img src="https://upload.wikimedia.org/wikipedia/commons/c/c7/HEIG-VD_Logo_96x29_RVB_ROUGE.png" alt="HEIG-VD Logo" width="250" /> 

# Cours TAL - Laboratoire 2
# Mise en œuvre et évaluation de *POS taggers* pour le français

**Objectif**

Appliquer des étiqueteurs morphosyntaxiques (POS taggers) disponibles dans NLTK et dans les outils Stanford NLP à des textes français, puis quantifier leurs performances.

**Instructions initiales**

* Télécharger l'archive `UD_French-GSD-withBlankLines.zip` fournie sur Cyberlearn.
* Placer les trois fichiers qu'elle contient dans le même dossier que le notebook.
* Ce sont des textes en français annotés avec les POS tags, provenant du projet ([Universal Dependencies](https://github.com/UniversalDependencies/UD_French-GSD)), et légèrement modifiés.
  - le fichier `fr-ud-train.conllu3` est destiné à l'entraînement
  - le fichier `fr-ud-dev.conllu3` est destiné aux tests préliminaires et aux réglages des paramètres
  - le fichier `fr-ud-test.conllu3` est destiné à l'évaluation finale.

**Questions préliminaires**

* En inspectant les fichiers, veuillez indiquer le numéro de la colonne où se trouvent les mots, et celui de la colonne où se trouvent leur étiquettes morpho-syntaxiques (*POS tags*).
* Veuillez chercher sur le Web la liste des *POS tags* du projet Universal Dependencies, avec leurs définitions, et indiquer l'URL ci-dessous.


In [2]:
# Veuillez écrire vos réponses dans cette cellule.
cols = {'0': 'Index', '1': 'Token', '2': 'Lemma', '3': 'POS'}
url_pos = "https://universaldependencies.org/u/pos/"
url_format = "https://universaldependencies.org/format.html"

print("Contenu des colonnes :")

for key, value in cols.items():
    print("     Colonne", key, ":", value)

print("Liste des POS TAGS du projet Universal Dependencies:")
print(url_pos)

print("Format des fichiers du projet Universal Dependencies:")
print(url_format)


Contenu des colonnes :
     Colonne 0 : Index
     Colonne 1 : Token
     Colonne 2 : Lemma
     Colonne 3 : POS
Liste des POS TAGS du projet Universal Dependencies:
https://universaldependencies.org/u/pos/
Format des fichiers du projet Universal Dependencies:
https://universaldependencies.org/format.html


* Veuillez déterminer et afficher le nombre de tokens de chacun des trois fichiers.

In [87]:
# Veuillez écrire votre code ci-dessous, puis exécuter cette cellule.
from nltk.corpus.reader import ConllCorpusReader

# Utilitaire pour afficher un titre
def print_title(str, char, size):
    print(f"{'':{char}^{size}}")
    print(f"{str:^{size}}")
    print(f"{'':{char}^{size}}")

# Fichiers de travail
file_location = 'UD_French-GSD-withBlankLines'
file_train = 'fr-ud-train.conllu3'
file_dev = 'fr-ud-dev.conllu3'
file_test = 'fr-ud-test.conllu3'

# Lecture des fichiers
columnsIgnore = ('ignore', 'words', 'ignore', 'pos')
train_corpus = ConllCorpusReader(file_location, file_train, columntypes=columnsIgnore, separator='\t')
dev_corpus = ConllCorpusReader(file_location, file_dev, columntypes=columnsIgnore, separator='\t')
test_corpus = ConllCorpusReader(file_location, file_test, columntypes=columnsIgnore, separator='\t')
corpuses = {'train': train_corpus, 'dev': dev_corpus, 'test': test_corpus}

print_title("Corpus d'entraînement", "_", 50)
print(f"    Il y a {len(corpuses['train'].words())} tokens dans ce corpus.")
print(f"    Il y a {len(corpuses['train'].sents())} phrases dans ce corpus.")
print_title("Corpus de développement", "_", 50)
print(f"    Il y a {len(corpuses['dev'].words())} tokens dans ce corpus.")
print(f"    Il y a {len(corpuses['dev'].sents())} phrases dans ce corpus.")
print_title("Corpus de test", "_", 50)
print(f"    Il y a {len(corpuses['test'].words())} tokens dans ce corpus.")
print(f"    Il y a {len(corpuses['test'].sents())} phrases dans ce corpus.")

__________________________________________________
              Corpus d'entraînement               
__________________________________________________
    Il y a 366371 tokens dans ce corpus.
    Il y a 14554 phrases dans ce corpus.
__________________________________________________
             Corpus de développement              
__________________________________________________
    Il y a 36830 tokens dans ce corpus.
    Il y a 1478 phrases dans ce corpus.
__________________________________________________
                  Corpus de test                  
__________________________________________________
    Il y a 10298 tokens dans ce corpus.
    Il y a 416 phrases dans ce corpus.


## Partie 1 : Évaluer le Stanford POS tagger avec les modèles fournis pour le français

L'Université de Stanford fournit un étiqueteur morpho-syntaxique (POS tagger) qui utilise l'apprentissage automatique (https://nlp.stanford.edu/software/tagger.html) appelé Maxent Tagger.  Le tagger et ses modèles multilingues peuvent être téléchargés à l'URL ci-dessus (archive ZIP suivant le lien *Download > full Stanford Tagger version 3.9.2*, 130 MB environ).  

Pour simplifier, on vous propose de télécharger séparément le programme Java [stanford-postagger.jar](https://drive.switch.ch/index.php/s/hMY6yO7lmoQJuS3) et le modèle français [french-ud.tagger](https://drive.switch.ch/index.php/s/4HSqKRTTTkCgPfB) fournis par l'enseignant (mot de passe = reference).  Enregistrez ces deux fichiers dans le même dossier que ce notebook.

Le Maxent Tagger est en Java, et peut être exécuté depuis ce notebook avec un appel Java en ligne de commande.  Pour exécuter une commande système depuis le notebook, ajouter '!' devant (par exemple `! dir` ou `! ls`).  Utilisez la [documentation du Maxent Tagger](https://nlp.stanford.edu/nlp/javadoc/javanlp/edu/stanford/nlp/tagger/maxent/MaxentTagger.html), et plus précisément la section *Tagging and Testing from the command line*, pour comprendre comment l'invoquer.  Java doit être installé sur votre système, et si nécessaire, exécuter :
```python
import os
java_path = 'C:/Program Files (x86)/Java/jdk1.8.0_20/bin/java.exe'  # votre chemin de java.exe
os.environ['JAVA_HOME'] = java_path   # attention aux slash (pas backslash sous Windows)
```
*Note* : il est également possible d'appeler ce tagger avec des commandes NLTK grâce au module [nltk.tag.stanford](https://www.nltk.org/_modules/nltk/tag/stanford.html) mais la gestion des *paths* entre Java, les classes et les modèles peut être compliquée.

**Question**

Appliquez le Maxent Tagger pour étiqueter le fichier `fr-ud-dev.conllu3` et demandez à Maxent Tagger de mesurer la qualité par comparaison à une l'annotation de référence fournie dans le fichier. Quels sont les scores obtenus ?  Quel est le nombre le plus important?  Indiquez ces réponses en commentaires du code.

In [6]:
from nltk.tag import StanfordPOSTagger


tokens_dev = [word for word in dev_corpus.words()]
tokens_test = [word for word in test_corpus.words()]
tokens_train = [word for word in train_corpus.words()]

In [45]:
# Veuillez écrire votre code ci-dessous, puis exécuter cette cellule.
st = StanfordPOSTagger('french-ud.tagger', 'stanford-postagger.jar')

# demandez à Maxent Tagger de mesurer la qualité par comparaison à une l'annotation de référence fournie dans le fichier. 
pred = st.tag(tokens_dev)
y_true = corpuses['dev'].tagged_sents()


AttributeError: 'LazyMap' object has no attribute 'shape'

De même, appliquez le Maxent Tagger pour étiqueter le fichier `fr-ud-test.conllu3` et indiquez la précision du tagger en commentaires du code (#).

In [46]:
# Veuillez écrire votre code ci-dessous, puis exécuter cette cellule.
pred = st.tag(tokens_test)
y_true = corpuses['test'].tagged_sents()


**Question subsidiaire** : combien de phrases et de mots le tagger trouve-t-il dans les fichiers `fr-ud-dev.conllu3` et `fr-ud-test.conllu3` ?  Comparez avec votre propre estimation du nombre de mots.

In [76]:
# Veuillez écrire vos réponses ci-dessous, en commentaires
print_title("Corpus d'entraînement", "_", 50)
print(f"Il y a {len(corpuses['dev'].words())} mots dans le corpus de développement.")
print(f"Il y a {len(corpuses['dev'].sents())} phrases dans le corpus de développement.")
print_title("Corpus de test", "_", 50)
print(f"Il y a {len(corpuses['test'].words())} mots dans le corpus de test.")
print(f"Il y a {len(corpuses['test'].sents())} phrases dans le corpus de test.")


__________________________________________________
              Corpus d'entraînement               
__________________________________________________
Il y a 36830 mots dans le corpus de développement.
Il y a 1478 phrases dans le corpus de développement.
__________________________________________________
                  Corpus de test                  
__________________________________________________
Il y a 10298 mots dans le corpus de test.
Il y a 416 phrases dans le corpus de test.


## Partie 2 : Entraîner le Stanford POS tagger pour obtenir de nouveaux modèles

Le but de cette partie est d'entraîner le Maxent Tagger sur les données UD en français (`fr-ud-train.conllu3`), puis de comparer le modèle obtenu avec les modèles fournis par Stanford pour le français, testés dans la partie 1A.  

Suivre la [documentation de Maxent Tagger](https://nlp.stanford.edu/nlp/javadoc/javanlp/edu/stanford/nlp/tagger/maxent/MaxentTagger.html) pour l'entraîner sur le fichier `fr-ud-train.conllu3` et le tester sur `fr-ud-test.conllu3`.  Regardez la section *Training from the command line*. 

La configuration du système pour effectuer l'entraînement est donnée dans un fichier texte, qui peut être produit en suivant la documentation (option `-genprops` pour obtenir un template qui sera édité), soit en s'inspirant du fichier [french-ud.tagger.props](https://drive.switch.ch/index.php/s/gHlam9S74HG2Q4X) accompagnant le modèle `french-ud.tagger` que vous avez utilisé ci-dessus.  Pensez à donner un nouveau nom à votre fichier modèle.

**Questions**

* Créez un fichier `myFrench-ud.tagger.props` qui aboutit à un bon entraînement.  Vous pourrez expérimenter plusieurs fois et proposer le meilleur fichier.  Citez dans le notebook les paramètres sur lesquels vous avez agi.

* Lancez l'entraînement sur le fichier `fr-ud-train.conllu3` (s'il ne tient pas en mémoire, utilisez seulement `fr-ud-dev.conllu3`). Pendant l’entraînement (> 10 minutes, 500 itérations), regardez la suite du travail.

* Évaluez votre modèle comme ci-dessus (sur `dev` et sur `test`).  Quel modèle est meilleur, le vôtre ou celui fourni par Stanford ?  Formulez une hypothèse expliquant ce résultat. 

In [ ]:
# Veuillez écrire votre code ci-dessous, puis exécuter cette cellule.
! java \
    -cp stanford-postagger.jar     edu.stanford.nlp.tagger.maxent.MaxentTagger \
    -props labo_hyper_parameter_1.props \
    -nthreads 8

In [ ]:
! java \
    -cp stanford-postagger.jar     edu.stanford.nlp.tagger.maxent.MaxentTagger \
    -props labo_hyper_parameter_2.props \
    -nthreads 8

In [ ]:
# Veuillez écrire votre code ci-dessous, puis exécuter cette cellule.
# mx1g = 1 gigabyte de mémoire allouée
! java -mx1g \ 
    -cp edu.stanford.nlp.tagger.maxent.MaxentTagger \
    -model experiment1.tagger \
    -testFile UD_French-GSD_withBlankLines/fr-ud-dev.conllu3 \ 

In [ ]:
! java -mx1g \ 
    -cp edu.stanford.nlp.tagger.maxent.MaxentTagger \
    -model experiment2.tagger \
    -testFile UD_French-GSD_withBlankLines/fr-ud-dev.conllu3 \ 

## Partie 3 : entraîner un POS tagger pour le français dans NLTK

Le but de cette partie est d'utiliser le POS tagger *Averaged Perceptron* de NLTK, en l'entraînant pour le français sur les mêmes données que ci-dessus.  

Notez que pour l'anglais, des taggers pré-entraînés sont disponibles dans NLTK, comme expliqué au [Chapitre 5.1 du livre NLTK](http://www.nltk.org/book/ch05.html) : on peut écrire `nltk.pos_tag(sentence)` où *sentence* est une phrase tokenisée. L'étiquetage morpho-syntaxique produira des paires ('mot', 'TAG').

**Première étape**

Importer les textes annotés `fr-ud-XXXX.conllu3` grâce à des objets `ConllCorpusReader`.  Consultez le mode d'emploi de cette classe directement dans [son code source](https://www.nltk.org/_modules/nltk/corpus/reader/conll.html#ConllCorpusReader), pour déterminer comment lire un fichier en créant un objet `ConllCorpusReader`.  Chargez les trois fichiers, dans trois objets appelés `train_corpus`, `dev_corpus` et `test_corpus`.

In [ ]:
from nltk.corpus.reader.conll import ConllCorpusReader

In [ ]:
# Veuillez écrire votre code ci-dessous, puis exécuter cette cellule.
train_corpus = ConllCorpusReader("UD_French-GSD-withBlankLines", "fr-ud-train.conllu3", columntypes=('ignore', 'words', 'ignore', 'pos'), separator='\t')
dev_corpus = ConllCorpusReader("UD_French-GSD-withBlankLines", "fr-ud-dev.conllu3", columntypes=('ignore', 'words', 'ignore', 'pos'), separator='\t')
test_corpus = ConllCorpusReader("UD_French-GSD-withBlankLines", "fr-ud-test.conllu3", columntypes=('ignore', 'words', 'ignore', 'pos'), separator='\t')
corpuses = {'train': train_corpus, 'dev': dev_corpus, 'test': test_corpus}

Affichez le nombre de phrases et le nombre de mots de chaque corpus chargé. Cesc chiffres sont-ils identiques à ceux obtenus pour `dev`et pour `test` à la fin de la Partie 1 ?  On peut obtenir les listes de mots étiquetés avec `tagged_words()` et les listes de phrases avec mots étiquetés avec `tagged_sents()`.

In [ ]:
# Veuillez écrire votre code ci-dessous, puis exécuter cette cellule.



for corpus_name, corpus in corpuses.items():
    print(f"{corpus_name} : {len(corpus.tagged_sents())} phrases, {len(corpus.tagged_words())} mots")

Affichez la 17e phrase du corpus de développement (avec les étiquettes POS), et les mots 1001 à 1050 du corpus de test (aussi avec leurs POS tags).

In [ ]:
# Veuillez écrire votre code ci-dessous, puis exécuter cette cellule.

print("17e phrase du corpus de développement :")
print(train_corpus.tagged_sents()[16])

print("1001e à 1050e mots du corpus de test :")
print(test_corpus.tagged_words()[1001:1050])

**Seconde étape**

Vous allez maintenant entraîner (sur le corpus `train`) le POS tagger appelé *Averaged Perceptron* fourni par NLTK mais [implémenté par Mathew Honnibal de Explosion.AI](https://explosion.ai/blog/part-of-speech-pos-tagger-in-python).

Dans le [package de NLTK avec des taggers](http://www.nltk.org/api/nltk.tag.html), considérez le module `nltk.tag.perceptron`, pour lequel NLTK explique de façon précise l'entraînement (voir *train the model*) et le test.  Vous allez mettre en oeuvre ces étapes pour entraîner le tagger.  Notez que le modèle est enregistré dans un fichier qui doit finir par `.pickle`, et qui est écrasé à chaque entraînement si vous ne changez pas de nom.  Un modèle peut être également chargé dans un tagger.

In [ ]:
# import os # si nécessaire
# import nltk # si nécessaire
# nltk.download('averaged_perceptron_tagger') # si nécessaire
from nltk.tag.perceptron import PerceptronTagger

In [ ]:
ptagger = PerceptronTagger(load=False)

Entraînez ici le tagger sur les données d'entraînement, avec les meilleurs paramètres possibles.

In [ ]:
MODEL_FILE = "perceptron_model.pickle"

In [ ]:
# Veuillez écrire votre code ci-dessous, puis exécuter cette cellule.
%%timeit
ptagger.train(train_corpus.tagged_sents(), MODEL_FILE, nr_iter=5)

# TODO : demander si il y a d'autres hyper paramètres que nr_iter

Combien de temps prend l'entraînement ?  Quelle est la taille du fichier modèle résultant ?

In [ ]:
# Veuillez écrire vos réponses dans cette cellule (en commentaires).
# Avec 5 itérations, l'entraînement prend environ 1 minute
# Le modèle fait 8 Mo
! du -h perceptron_model.pickle

Évaluez le tagger, d'abord sur les données `dev` puis sur les données `test`.

In [ ]:
# Veuillez écrire votre code ci-dessous, puis exécuter cette cellule.


ptagger.load(MODEL_FILE)
prediction = ptagger.tag(test_corpus.words())

# https://www.nltk.org/api/nltk.tag.api.html


In [ ]:

print(f"Matrice de confusion et accuracy sur le corpus de dev : ")
print(ptagger.confusion(dev_corpus.tagged_sents()))
print(ptagger.accuracy(dev_corpus.tagged_sents()))



In [ ]:
print(f"Matrice de confusion et accuracy sur le corpus de test : ")
print(ptagger.confusion(test_corpus.tagged_sents()))
print(ptagger.accuracy(test_corpus.tagged_sents()))

Veuillez remplir le tableau suivant avec la synthèse des résultats.

| Corpus | MaxEnt | MaxEnt   | Avg Perceptron | 
|--------|--------|----------|---------------|
| -      | fourni | entraîné | entraîné |
| dev    |   ..   |   ..     |  ..  |
| test   |   ..   |   ..     |  ..  |

Comment se comparent les deux POS taggers sur le français ?  Écrivez vos conclusions dans cette cellule.

## Fin du laboratoire 2  

Merci de nettoyer votre feuille, exécuter une dernière fois toutes les instructions, sauvegarder le résultat, et le rendre via Cyberlearn.